# Challenge
Identify the top 5 members in terms of booking frequency.

To identify the top 5 members in terms of booking frequency, focus on the bookings table. This table contains all the information you need to determine how frequently each member has made bookings. Remember, aggregation and sorting will be key techniques in this challenge.

## Extract 
First I'l load the required dependencies and establish the connection parameters for the remote transactional database. 

In [34]:
import psycopg2
import pandas as pd

# The connection details (user, password, host) below are placeholders.
# Your coach will provide you with the actual connection details for your exercises.

conn_params = {
    'host': 'etl-relational-database.cfmnnswnfhpn.eu-west-2.rds.amazonaws.com',
    'dbname': 'postgres',
    'user': 'readonly_user',
    'password': 'you-can-only-read'
}

Next, define the function that will be used to fetch data from the remote DB. 

In [35]:
def fetch_data(query):
    with psycopg2.connect(**conn_params) as conn:
        with conn.cursor() as cur:
            cur.execute(query)
            data = cur.fetchall()
            colnames = [desc[0] for desc in cur.description]
    return data, colnames

Then fetch the necessary data and create a pandas dataframe with it. 

As the member id isn't particularly useful in answering the question on it's own I will join the members and bookings tables to get the first and last names of the top 5 members, rather than just their member id's. 

In [36]:
# Extract bookings data
query = "SELECT b.memid, m.surname, m.firstname, COUNT(b.memid) as count FROM bookings b JOIN members m ON b.memid = m.memid GROUP BY b.memid, m.surname, m.firstname ORDER BY count DESC LIMIT 5"
bookings_data, colnames = fetch_data(query)
bookings_df = pd.DataFrame(bookings_data, columns=colnames)
bookings_df

,memid,surname,firstname,count
0,0,GUEST,GUEST,883
1,3,Rownam,Tim,408
2,1,Smith,Darren,261
3,2,Smith,Tracy,210
4,8,Boothe,Tim,188


## Transform
My personal preference is to use SQL to directly extract the required data. This minimises the number of steps required to transform the data and streamlines the process slightly. 

However, this would only really be appropriate if I knew that only one particular transformation of the data is needed. If multiple different transformations were required, or if I wasn't certain that I only needed one specific transformation, it might be better to use a broader SQL query and to manipulate the data as required with Pandas.

This way I could just query the remote database once and conduct multiple transformations on the result if needed. I've show below how I would go about doing this. 

In [37]:
# query = "SELECT * FROM bookings b JOIN members m USING (memid)"

# bookings_data, colnames = fetch_data(query)
# bookings_df = pd.DataFrame(bookings_data, columns=colnames)

# # Grouping by memid, surname, and firstname and counting occurrences
# grouped_data = bookings_df.groupby(['memid', 'surname', 'firstname']).size().reset_index(name='count')

# # Sorting the data by count in descending order and taking the top 5
# top_5 = grouped_data.sort_values(by='count', ascending=False).head(5)

# top_5


## Load
Once I have th necessary data in a dataframe I then need to insert that data into my local analytical database. 

In the cell below I set up the connection to my analytical database and then execute to query to create a table which I will populate with my transformed data. 

In [38]:
def execute_query_postgresql(conn_string, query):
    with psycopg2.connect(conn_string) as conn:
        with conn.cursor() as cur:
            cur.execute(query)
            conn.commit()

# Define the connection string for your analytical database
etl_bites_conn_string = "dbname='etl_bites' user='jackdench' host='localhost' port='5432'"

# SQL query to create a new table for storing total booking duration per facility
create_total_bookings_table = '''
CREATE TABLE top_5_members (
    MemberId INTEGER NOT NULL,
    MemberSurname TEXT,
    MemberFirstname TEXT,
    BookingsCount INTEGER
);
'''

# Execute the query to create the table
execute_query_postgresql(etl_bites_conn_string, create_total_bookings_table)

Finally, I will execute the INSERT query to populate the analytical database with my transformed data. 

In [39]:
def insert_data(conn_string, table_name, data, columns):
    with psycopg2.connect(conn_string) as conn:
        with conn.cursor() as cur:
            for row in data.itertuples(index=False):
                insert_query = f"INSERT INTO {table_name} ({', '.join(columns)}) VALUES ({', '.join(['%s'] * len(columns))});"
                cur.execute(insert_query, row)
            conn.commit()

# Insert the transformed data into the analytical database
insert_data(etl_bites_conn_string, 'top_5_members', bookings_df, ['MemberId', 'MemberSurname', 'MemberFirstname', 'BookingsCount'])

I have included a screenshot from TablePlus in my submission showing the data successfully loaded there.